# Parameters and Parameter Space

This notebook demonstrates usage of the Parameter and ParameterSpace classes in python. These classes can work standalone within python, as shown here, but they can also connect to a TINC server through the TincClient class.

In [ ]:
from tinc import *
TincVersion()

## Parameters

Parameters must have a unique name passed to the constructor

In [ ]:
p = Parameter("temperature")

Parameters have a current value, that you can read and write to using the 'value' member

In [ ]:
p.value

In [ ]:
p.value = 30

In [ ]:
p.value

### Boundaries

You can set the boundaries of the parameter using 'minimum'and 'maximum

In [ ]:
p.minimum = 10
p.maximum = 30

In [ ]:
p.value = 0
p.value

In [ ]:
p.value = 32
p.value

## Defining possible values for parameter/dimension

You can set the possible values a parameter can take:

In [ ]:
import numpy as np

In [ ]:
p.values = np.linspace(10, 40, 10)

In [ ]:
p.values

In [ ]:
p.value = 50

Incoming values are clamped to the boundaies set by the valid values and rounded to the nearest possible value

In [ ]:
p.value

In [ ]:
p.value = 31
p.value

In [ ]:
p.value = 32
p.value

### Value callbacks

You can register a callback function that can be triggered whenever a parameter's value changes. The callback function gets passed the new parameter value

In [ ]:
def computation(value):
    print(f'Got {value}')

p.register_callback(computation)

In [ ]:
p.value = 70

### Value ids

Each possible value the parameter can take can have an associated id. This can be useful for example to map parameter to filesystem names.

In [ ]:
chem_pot = Parameter("chemical_potential")
chem_pot.set_values([-0.3, -0.15, 0.0, 0.15, 0.3, 0.45])
chem_pot.set_ids(['-0.30', '-0.15', '0', '0.15', '0.30', '0.45'])

In [ ]:
chem_pot.value = -0.15
chem_pot.get_current_id()

In [ ]:
chem_pot.value = -0.14
chem_pot.get_current_id()

## Parameter Spaces

A ParameterSpace in TINC is a class that groups and manages parameters. In that sense, each parameter is a dimension of the parameter space.

In [ ]:
ps = ParameterSpace("ps1")

In [ ]:
ps.register_parameters([p,chem_pot])

A parameter space can be associated with a filesystem path that can change according to its parameters

In [ ]:
ps.set_current_path_template("t_%%temperature%%_chempot_%%chemical_potential:ID%%")

In [ ]:
ps.get_current_relative_path()

In [ ]:
chem_pot.value = 0.45
ps.get_current_relative_path()

You can execute a function through the parameter space to have it be run with the parameter space's current values:

In [ ]:
def processor(temperature, chemical_potential):
    print(f"Running at: T {temperature} -- chem_pot {chemical_potential}")

ps.run_process(processor)

In [ ]:
p.value = 30
ps.run_process(processor)

You can have the parameter space cache the output for these processes automatically, by enabling caching. You can determine where this cache gets stored by passing the foler name.

In [ ]:
ps.enable_cache()

In [ ]:
import time

def long_processor(temperature, chemical_potential):
    print(f"Running at: T {temperature} -- chem_pot {chemical_potential}")
    time.sleep(3) # Simulate a time consuming process
    return temperature * chemical_potential

In [ ]:
%%time
value = ps.run_process(long_processor)

In [ ]:
value

In [ ]:
%%time
value = ps.run_process(long_processor)
value

In [ ]:
p.value = 10

In [ ]:
%%time
value = ps.run_process(long_processor)
value

In [ ]:
%%time
value = ps.run_process(long_processor)
value

In [ ]:
p.value = 30

In [ ]:
%%time
value = ps.run_process(long_processor)
value

### Parameter sweeps

You can run parameter sweeps generating cache:

In [ ]:
%%time
ps.sweep(long_processor)